# Week 1: Programming Assignment
## Imports

In [1]:
import re

import pulp as pl

## Problem 1: Use PuLP to encode a linear programming problem

In [2]:
# Define a key function to extract the numeric part of each variable name
def extract_number(lp_var):
    match_obj = re.search(r"\d+", lp_var.name)
    assert match_obj is not None
    return int(match_obj.group())


def formulate_lp_problem(m, n, list_c, list_a, list_b):
    # Assert that the data is compatible
    assert m > 0
    assert n > 0
    assert len(list_c) == n
    assert len(list_a) == m
    assert len(list_a) == len(list_b)
    assert all(len(lst) == n for lst in list_a)

    # Create a linear programming model and set it to maximize its objective
    lp_model = pl.LpProblem("LPProblem", pl.LpMaximize)

    # Create all the decision variables and store them in a list
    decision_vars = [pl.LpVariable(f"x{i}") for i in range(n)]

    # Create the objective function
    lp_model += pl.lpSum([c * v for c, v in zip(list_c, decision_vars)])

    # Create all the constraints
    for coeffs, rhs in zip(list_a, list_b):
        lhs = pl.lpSum([c * v for c, v in zip(coeffs, decision_vars)])
        lp_model += lhs <= rhs

    # Solve the problem and get its status
    lp_model.solve()
    status = pl.LpStatus[lp_model.status]

    # Return the expected tuple
    is_feasible = False
    is_bounded = False
    opt_sol = []

    if status == "Optimal":
        is_feasible = True
        is_bounded = True
        # Get the model variables in the right order
        lp_vars = sorted(lp_model.variables(), key=extract_number)
        opt_sol = [lp_var.varValue for lp_var in lp_vars]
    elif status == "Unbounded":
        is_feasible = True

    return is_feasible, is_bounded, opt_sol

In [3]:
# Test 1
m = 4
n = 3
list_c = [1, 1, 1]
list_a = [[2, 1, 2], [1, 0, 0], [0, 1, 0], [0, 0, -1]]
list_b = [5, 7, 9, 4]
is_feas, is_bnded, sols = formulate_lp_problem(m, n, list_c, list_a, list_b)
assert is_feas, "The LP should be feasible -- your code returns infeasible"
assert is_bnded, "The LP should be bounded -- your code returns unbounded"
print(sols)
assert abs(sols[0] - 2.0) <= 1e-04, "x0 must be 2.0"
assert abs(sols[1] - 9.0) <= 1e-04, "x1 must be 9.0"
assert abs(sols[2] + 4.0) <= 1e-04, "x2 must be -4.0"
print("Passed: 3 points!")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/woitek/.local/share/virtualenvs/coursera_linear_programming-sQGDHXUS/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4b73ed8cf5f44927896ad8c2d4635e1c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/4b73ed8cf5f44927896ad8c2d4635e1c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 19 RHS
At line 24 BOUNDS
At line 28 ENDATA
Problem MODEL has 4 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-4) rows, 0 (-3) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 7
After Postsolve, objective 7, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 7 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):  

In [4]:
# Test 2: Unbounded problem
m = 5
n = 4
list_c = [-1, 2, 1, 1]
list_a = [[1, 0, -1, 2], [2, -1, 0, 1], [1, 1, 1, 1], [1, -1, 1, 1], [0, -1, 0, 1]]
list_b = [3, 4, 5, 2.5, 3]
is_feas, is_bnded, sols = formulate_lp_problem(m, n, list_c, list_a, list_b)
assert is_feas, "The LP should be feasible. But your code returns a status of infeasible."
assert not is_bnded, "The LP should be unbounded but your code returns a status of bounded."
print("Passed: 3 points")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/woitek/.local/share/virtualenvs/coursera_linear_programming-sQGDHXUS/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4bfbb5d0b389431b93e223da028387d7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/4bfbb5d0b389431b93e223da028387d7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 31 RHS
At line 37 BOUNDS
At line 42 ENDATA
Problem MODEL has 5 rows, 4 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve thinks problem is unbounded
Analysis indicates model infeasible or unbounded
0  Obj -0 Dual inf 0.0499996 (4) w.o. free dual inf (0)
0  Obj -0 Dual inf 0.0499996 (4) w.o. free dual inf (0)
1  Obj 10 Dual inf 0.0499997 (3) w.o. free dual inf (0)
1  Obj 10 Dual inf 0.0499997 (3) w.o. free dual inf (0)
Dual infeasible - objective value 10


In [5]:
# Test 3: Infeasible problem
m = 4
n = 3
list_c = [1, 1, 1]
list_a = [[-2, -1, -2], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
list_b = [-8, 1, 1, 1]
is_feas, is_bnded, sols = formulate_lp_problem(m, n, list_c, list_a, list_b)
assert not is_feas, "The LP should be infeasible -- your code returns feasible"
print("Passed: 3 points!")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/woitek/.local/share/virtualenvs/coursera_linear_programming-sQGDHXUS/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ca28a63cf43c48c6ad75d10db43dc658-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/ca28a63cf43c48c6ad75d10db43dc658-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 19 RHS
At line 24 BOUNDS
At line 28 ENDATA
Problem MODEL has 4 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 7.9999999 (1) Dual inf 0.0299997 (3) w.o. free dual inf (0)
3  Obj 4.5 Primal inf 1.4999999 (1) Dual inf 0.4999999 (1)
4  Obj 3 Primal inf 2.9999999 (1)
Primal infeasible - objective value 3
PrimalInfeas

In [6]:
# Test 4
m = 16
n = 15
list_c = [1] * n
list_c[6] = list_c[6] + 1
list_a = []
list_b = []
for i in range(n):
    lst = [0] * n
    lst[i] = -1
    list_a.append(lst)
    list_b.append(0)
list_a.append([1] * n)
list_b.append(1)
is_feas, is_bnded, sols = formulate_lp_problem(m, n, list_c, list_a, list_b)
assert is_feas, "Problem is feasible but your code returned infeasible"
assert is_bnded, "Problem is bounded but your code returned unbounded"
print(sols)
assert abs(sols[6] - 1.0) <= 1e-03, "Solution does not match expected one"
assert all([abs(sols[i]) <= 1e-03 for i in range(n) if i != 6]), "Solution does not match expected one"
print("Passed: 3 points!")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/woitek/.local/share/virtualenvs/coursera_linear_programming-sQGDHXUS/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8e77d1e59ac545199ab1d535a6c59651-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8e77d1e59ac545199ab1d535a6c59651-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 67 RHS
At line 84 BOUNDS
At line 100 ENDATA
Problem MODEL has 16 rows, 15 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-16) rows, 0 (-15) columns and 0 (-30) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 2
After Postsolve, objective 2, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU sec